# IMPORT DATA AND REMOVE OUTLIERS

## Import Data:

In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import warnings

warnings.filterwarnings('ignore')

df_original = pd.read_csv('result/data/melting_point_features.csv')

## REMOVE OUTLIERS

In [14]:
print("--- BẮT ĐẦU XỬ LÝ & CHIA DỮ LIỆU ---")

df_clean = df_original.dropna(subset=['Tm'])
df_clean = df_clean[(df_clean['Tm'] > 0) & (df_clean['Tm'] < 4000)].copy()

print(f"1. Dữ liệu sau khi lọc: {len(df_clean)} dòng (Đã loại {len(df_original) - len(df_clean)} nhiễu)")

y = df_clean['Tm']
X = df_clean.drop(columns=['Tm', 'SMILES'], errors='ignore').select_dtypes(include=[np.number])

X.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='median')
imputer.fit(X)
X_final = pd.DataFrame(imputer.transform(X), columns=X.columns)

joblib.dump(imputer, 'result/imputer.pkl')

y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, 
    y_log, 
    test_size=0.2, 
    random_state=2601
)

y_test_real = np.expm1(y_test)
joblib.dump(imputer, 'imputer.pkl')

print("-" * 30)
print(f"✅ HOÀN TẤT! Dữ liệu đã sạch và được chia:")
print(f"   - X_train: {X_train.shape} | y_train (Log): {y_train.shape}")
print(f"   - X_test : {X_test.shape}  | y_test  (Log): {y_test.shape}")

--- BẮT ĐẦU XỬ LÝ & CHIA DỮ LIỆU ---
1. Dữ liệu sau khi lọc: 8766 dòng (Đã loại 1754 nhiễu)
------------------------------
✅ HOÀN TẤT! Dữ liệu đã sạch và được chia:
   - X_train: (7012, 937) | y_train (Log): (7012,)
   - X_test : (1754, 937)  | y_test  (Log): (1754,)


# MODEL TRAINING: FEATURES SELECTION USING RFECV, GA 

## RFECV Train

In [15]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
import joblib
import numpy as np

print("\n--- 2. CHẠY RFECV ---")
start = time.time()

model_rfe = LGBMRegressor(
    objective='regression',
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=2601,
    n_jobs=-1, verbose=-1
)

rfe = RFECV(
    estimator=model_rfe,
    min_features_to_select=50,
    step=20,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

rfe.fit(X_train, y_train)

selected_rfe = X_train.columns[rfe.support_]
print(f"⏱️ Time Run: {time.time() - start:.2f} s")
print(f"✅ RFECV Selected: {len(selected_rfe)} features")


print("\n--- 3. ĐÁNH GIÁ ---")

eval_model = LGBMRegressor(
    n_estimators=3000, learning_rate=0.01, num_leaves=50, max_depth=-1,
    subsample=0.8, colsample_bytree=0.7, random_state=2601, n_jobs=-1, verbose=-1
)

eval_model.fit(X_train[selected_rfe], y_train)

y_pred_log = eval_model.predict(X_test[selected_rfe])
y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n" + "="*40)
print("📊 KẾT QUẢ RFECV (REAL SCALE)")
print(f"Features : {len(selected_rfe)}")
print(f"MAE      : {mae:.4f} K")
print(f"RMSE     : {rmse:.4f} K")
print(f"R2       : {r2:.4f}")
print("="*40)

joblib.dump(eval_model, 'result/melting_point_model_rfecv.pkl')
joblib.dump(list(selected_rfe), 'result/features_list_rfecv.pkl')


--- 2. CHẠY RFECV ---
Fitting estimator with 937 features.
Fitting estimator with 917 features.
Fitting estimator with 897 features.
Fitting estimator with 877 features.
Fitting estimator with 857 features.
Fitting estimator with 837 features.
Fitting estimator with 817 features.
Fitting estimator with 797 features.
Fitting estimator with 777 features.
Fitting estimator with 757 features.
Fitting estimator with 737 features.
Fitting estimator with 717 features.
Fitting estimator with 697 features.
Fitting estimator with 677 features.
Fitting estimator with 657 features.
Fitting estimator with 637 features.
Fitting estimator with 617 features.
Fitting estimator with 597 features.
Fitting estimator with 577 features.
Fitting estimator with 557 features.
⏱️ Time Run: 543.84 s
✅ RFECV Selected: 537 features

--- 3. ĐÁNH GIÁ ---

📊 KẾT QUẢ RFECV (REAL SCALE)
Features : 537
MAE      : 83.3528 K
RMSE     : 193.6333 K
R2       : 0.5936


['result/features_list_rfecv.pkl']

## GENETIC ALGORITHM (GA) Train

In [ ]:
from sklearn_genetic import GAFeatureSelectionCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
import joblib
import numpy as np

print("\n--- 2. CHẠY GENETIC ALGORITHM (GA) ---")
start = time.time()

model_ga = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31, 
    ax_depth=-1,
    random_state=2601,
    n_jobs=1, verbose=-1
)

ga = GAFeatureSelectionCV(
    estimator=model_ga,
    cv=3,
    scoring="neg_mean_absolute_error",
    population_size=50,
    generations=15,
    mutation_probability=0.1,
    crossover_probability=0.8,
    keep_top_k=2,
    elitism=True,
    n_jobs=-1, verbose=True
)

ga.fit(X_train, y_train)

selected_ga = X_train.columns[ga.support_]

print(f"Time Run: {time.time() - start:.2f} s")
print(f"\n✅ GA Chosen {len(selected_ga)} features")
joblib.dump(list(selected_ga), 'result/ga_features.pkl')

print("\n--- 3. ĐÁNH GIÁ (EVALUATION) ---")
eval_model = LGBMRegressor(
    n_estimators=3000, learning_rate=0.01, num_leaves=50, max_depth=-1,
    subsample=0.8, colsample_bytree=0.7, random_state=2601, n_jobs=-1, verbose=-1
)

eval_model.fit(X_train[selected_ga], y_train)

y_pred_log = eval_model.predict(X_test[selected_ga])

y_pred_real = np.expm1(y_pred_log)

y_test_real = np.expm1(y_test)

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n--- RESULT GA (REAL SCALE) ---")
print(f"Features: {len(selected_ga)}")
print(f"MAE     : {mae:.4f} K")
print(f"RMSE    : {rmse:.4f} K")
print(f"R2      : {r2:.4f}")

joblib.dump(model_ga, 'result/melting_point_model_ga.pkl')
joblib.dump(list(selected_ga), 'result/features_list_ga.pkl')

# UNION AND INTERSECTION 2 MODEL TRAINNING

## FEATURES UNION 

In [ ]:
import joblib

rfe_features = joblib.load('result/rfe_features.pkl')
ga_features = joblib.load('result/ga_features.pkl')

common_features_uni = set(rfe_features) | set(ga_features)

print(f"\n💎 Tổng features sau khi hợp (Union): {len(common_features_uni)}")
print(list(common_features_uni))

In [ ]:
import joblib
from lightgbm import LGBMRegressor

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_log, test_size=0.2, random_state=2601
)

features_uni = list(common_features_uni)
valid_features_uni = [f for f in features_uni if f in X_final.columns]

manual_params = {
    'n_estimators': 8000,
    'learning_rate': 0.01,
    'num_leaves': 40,
    'max_depth': 15,
    'min_child_samples': 20,
    
    'objective': 'regression',
    'metric': 'rmse',
    'subsample': 0.7,
    'colsample_bytree': 0.6,
    
    'reg_alpha': 0.2,
    'reg_lambda': 0.2,
    
    'random_state': 2601,
    'n_jobs': -1,
    'verbose': -1
}

model_uni = LGBMRegressor(**manual_params)
model_uni.fit(X_train[valid_features_uni], y_train)

joblib.dump(model_uni, 'result/melting_point_model_uni.pkl')
joblib.dump(valid_features_uni, 'result/features_list_uni.pkl')

print("💾 Đã lưu model và features được Union thành công!")

## Score Union Features

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = joblib.load('result/melting_point_model_uni.pkl')
features = joblib.load('result/features_list_uni.pkl')

print(f"Đã load model. Đang đánh giá trên tập Test có ({len(X_test)} mẫu)...")

y_pred_raw = model.predict(X_test[features])

if y_test.max() > 15:
    y_test_real_eval = y_test
    print("Phát hiện y_test là thang đo Thực (giữ nguyên).")
else:
    y_test_real_eval = np.expm1(y_test)
    print("Phát hiện y_test là thang đo Log (đã chuyển về Thực).")

if y_pred_raw.max() > 15:
    y_pred_real = y_pred_raw
else:
    y_pred_real = np.expm1(y_pred_raw)

mae = mean_absolute_error(y_test_real_eval, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real_eval, y_pred_real))
r2 = r2_score(y_test_real_eval, y_pred_real)

print("\n--- RESULT UNION ---")
print(f"MAE : {mae:.4f} K")
print(f"RMSE: {rmse:.4f} K")
print(f"R2  : {r2:.4f}")

## FEATURES INTERSECTION 

In [ ]:
import joblib

rfe_features = joblib.load('result/rfe_features.pkl')
ga_features = joblib.load('result/ga_features.pkl')

common_features_int = set(rfe_features) & set(ga_features)

print(f"\n💎 Tổng features sau khi giao (Intersection): {len(common_features_int)}")
print(list(common_features_int))

In [ ]:
import joblib
from lightgbm import LGBMRegressor

best_features_int = list(common_features_int)
valid_features_int = [f for f in best_features_int if f in X_final.columns]

manual_params = {
    'n_estimators': 8000,
    'learning_rate': 0.01,
    'num_leaves': 40,
    'max_depth': 15,
    'min_child_samples': 20,
    'objective': 'regression',
    'metric': 'rmse',
    'subsample': 0.7,
    'colsample_bytree': 0.6,
    'reg_alpha': 0.2,
    'reg_lambda': 0.2,
    'random_state': 2601,
    'n_jobs': -1,
    'verbose': -1
}

model_int = LGBMRegressor(**manual_params)

model_int.fit(X_train[valid_features_int], y_train)

joblib.dump(model_int, 'result/melting_point_model_int.pkl')
joblib.dump(valid_features_int, 'result/features_list_int.pkl')

print(f"💾 Đã lưu Intersection ({len(valid_features_int)} features) thành công!")

## Score Intersection Features

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

model_int = joblib.load('result/melting_point_model_int.pkl')
features_int = joblib.load('result/features_list_int.pkl')

print(f"Đã load Model Intersection. Đang đánh giá trên tập Test có ({len(X_test)} mẫu)...")

y_pred_raw = model_int.predict(X_test[features_int])

if y_test.max() > 15:
    y_test_real_eval = y_test
    print("Phát hiện y_test là thang đo Thực.")
else:
    y_test_real_eval = np.expm1(y_test)
    print("Phát hiện y_test là thang đo Log (đã chuyển về Thực).")

if y_pred_raw.max() > 15:
    y_pred_real = y_pred_raw
else:
    y_pred_real = np.expm1(y_pred_raw)


mae = mean_absolute_error(y_test_real_eval, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real_eval, y_pred_real))
r2 = r2_score(y_test_real_eval, y_pred_real)

print("\n--- KẾT QUẢ INTERSECTION ---")
print(f"MAE : {mae:.4f} K")
print(f"RMSE: {rmse:.4f} K")
print(f"R2  : {r2:.4f}")

# Score each Method

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings

warnings.filterwarnings('ignore')

def get_metrics(name, feature_list):
    valid_feats = [f for f in feature_list if f in X_train.columns]

    if not valid_feats: 
        return {"Method": name, "Features": 0, "R2": 0, "MAE": 0, "RMSE": 0}

    model = LGBMRegressor(
        n_jobs=-1, verbose=-1, random_state=2601,
        n_estimators=8000, learning_rate=0.01,
        num_leaves=45, max_depth=-1,
        subsample=0.8, objective = 'regression_l1', metric= 'mae',
        colsample_bytree=0.7, reg_alpha=0.1, reg_lambda=0.1
    )

    callbacks = [lgb.early_stopping(stopping_rounds=1000, verbose=False)]

    model.fit(X_train[valid_feats], y_train, 
              eval_set=[(X_test[valid_feats], y_test)],
              eval_metric='rmse', callbacks=callbacks)

    y_pred_real = np.expm1(model.predict(X_test[valid_feats], num_iteration=model.best_iteration_))
    
    if y_test.max() > 15:
         y_test_real = y_test
    else:
         y_test_real = np.expm1(y_test)

    return {
        "Method": name, 
        "Features": len(valid_feats),
        "R2": r2_score(y_test_real, y_pred_real),
        "MAE": mean_absolute_error(y_test_real, y_pred_real),
        "RMSE": np.sqrt(mean_squared_error(y_test_real, y_pred_real))
    }

feats_all = list(X_train.columns)

try: 
    feats_rfe = joblib.load('result/rfe_features.pkl')
except: 
    feats_rfe = []
    print("Không tìm thấy file RFE features")

try: 
    feats_ga = joblib.load('result/ga_features.pkl')
except: 
    feats_ga = []
    print("Không tìm thấy file GA features")

feats_uni = list(set(feats_rfe) | set(feats_ga))
feats_int = list(set(feats_rfe) & set(feats_ga))

results = []
results.append(get_metrics("Original", feats_all))

if feats_rfe: results.append(get_metrics("RFE", feats_rfe))
if feats_ga:  results.append(get_metrics("GA", feats_ga))
if feats_uni: results.append(get_metrics("Union (RFE|GA)", feats_uni))
if feats_int: results.append(get_metrics("Intersect (RFE&GA)", feats_int))

df_res = pd.DataFrame(results)
if not df_res.empty:
    base = df_res.iloc[0]
    df_res['Diff_R2'] = df_res['R2'] - base['R2']
    df_res['Diff_MAE'] = df_res['MAE'] - base['MAE']
    df_res['Diff_RMSE'] = df_res['RMSE'] - base['RMSE']
    
    print("\n" + "="*85)
    print("             BẢNG SO SÁNH HIỆU QUẢ CÁC PHƯƠNG PHÁP CHỌN FEATURES")
    print("="*85)
    print(df_res.round(4).to_string(index=False))
    print("="*85)

# ELBOW PLOT TRAIN MODEL

## Elbow GA Features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from lightgbm import LGBMRegressor

best_params = {
    'learning_rate': 0.05, 
    'n_estimators': 3000,
    'num_leaves': 100,
    'max_depth': 12,
    'min_child_samples': 10,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'reg_alpha': 0.0,
    'reg_lambda': 0.0,
    'random_state': 2601,
    'n_jobs': -1,
    'verbose': -1
}

valid_ga_feats = [f for f in list(selected_ga) if f in X_train.columns]

print("Đang xếp hạng features...")
ranker = LGBMRegressor(**best_params)
ranker.fit(X_train[valid_ga_feats], y_train)

imp_df = pd.DataFrame({
    'Feature': valid_ga_feats,
    'Importance': ranker.feature_importances_
}).sort_values(by='Importance', ascending=False)

sorted_feats = imp_df['Feature'].tolist()

steps = list(range(len(sorted_feats), 100, -20)) + list(range(100, 0, -5))
results = []

print(f"\nBắt đầu vòng lặp cắt giảm features ({len(steps)} vòng)...")

for k in steps:
    current_feats = sorted_feats[:k]
    
    model = LGBMRegressor(**best_params)
    model.fit(X_train[current_feats], y_train)
    
    y_pred_log = model.predict(X_test[current_feats])
    
    y_pred_real = np.expm1(y_pred_log)
    y_test_real = np.expm1(y_test)

    r2 = r2_score(y_test_real, y_pred_real)
    rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
    mae = mean_absolute_error(y_test_real, y_pred_real)

    print(f"   -> Dùng {k:3d} features: R2 = {r2:.4f} | MAE = {mae:.2f} | RMSE = {rmse:.2f}")
    results.append({'Num_Features': k, 'R2': r2, 'MAE': mae, 'RMSE': rmse})

df_results = pd.DataFrame(results).sort_values(by='Num_Features')
df_leaderboard = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)
csv_filename = 'result/feature_selection_results.csv'
df_results.to_csv(csv_filename, index=False)

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['R2'], marker='o', linewidth=2, color='blue')

best_row = df_results.loc[df_results['R2'].idxmax()]
plt.scatter(best_row['Num_Features'], best_row['R2'], color='red', s=150, zorder=5)
plt.annotate(f"Đỉnh: {best_row['R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['R2']), 
             xytext=(best_row['Num_Features']+20, best_row['R2']-0.01),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('Biểu đồ Elbow: Hiệu quả khi giảm dần số lượng Features', fontsize=14)
plt.xlabel('Số lượng Features', fontsize=12)
plt.ylabel('Độ chính xác (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_xaxis()
plt.show()

print("\nBẢNG XẾP HẠNG (SCORE GIẢM DẦN):")
print(df_leaderboard[['R2', 'Num_Features', 'MAE', 'RMSE']].head(10))

print("\nBẢNG THEO THỨ TỰ FEATURE (ÍT -> NHIỀU):")
print(df_results[['Num_Features', 'R2', 'MAE', 'RMSE']].head(10))

## Elbow RFECV Features

In [ ]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

print("\n--- START RFE WITH LEADERBOARD ---")
start = time.time()

model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=1,
    verbose=-1
)

rfecv = RFECV(
    estimator=model,
    step=20,
    cv=3,
    scoring='r2', 
    min_features_to_select=50,
    n_jobs=-1,
    verbose=1
)

rfecv.fit(X_train, y_train)

r2_scores = rfecv.cv_results_['mean_test_score']
n_scores = len(r2_scores)

feature_counts = [50 + i * 20 for i in range(n_scores)]

if len(feature_counts) > len(r2_scores):
    feature_counts = feature_counts[:len(r2_scores)]

df_results = pd.DataFrame({
    'Num_Features': feature_counts,
    'Score_R2': r2_scores
})

df_leaderboard = df_results.sort_values(by='Score_R2', ascending=False).reset_index(drop=True)

selected_rfecv = X_train.columns[rfecv.support_]
print(f"\nTime Run: {time.time() - start:.2f} s")
print(f"Best Number of Features: {rfecv.n_features_}")
print(f"Best CV R2 Score: {df_leaderboard.iloc[0]['Score_R2']:.4f}")

print("\nLEADERBOARD (DESCENDING SCORE):")
print(df_leaderboard.head(10).to_string(index=False))

print("\nPROGRESS (BY FEATURE COUNT):")
print(df_results.head(10).to_string(index=False))

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['Score_R2'], marker='o', color='green', linewidth=2)

best_row = df_leaderboard.iloc[0]
plt.scatter(best_row['Num_Features'], best_row['Score_R2'], color='red', s=150, zorder=5)
plt.annotate(f"Best: {best_row['Score_R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['Score_R2']), 
             xytext=(best_row['Num_Features']+20, best_row['Score_R2']-0.005),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('RFECV Performance Curve', fontsize=14)
plt.xlabel('Number of Features Selected', fontsize=12)
plt.ylabel('Cross Validation Score (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nSelected Features List:")
print(list(selected_rfecv))

y_pred_log = rfecv.predict(X_test)
y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

test_r2 = r2_score(y_test_real, y_pred_real)
test_mae = mean_absolute_error(y_test_real, y_pred_real)
test_rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("\n--- FINAL TEST EVALUATION (Best Features) ---")
print(f"R2 Score : {test_r2:.4f}")
print(f"MAE      : {test_mae:.4f} K")
print(f"RMSE     : {test_rmse:.4f} K")

df_results.to_csv('result/rfecv_results.csv', index=False)

# Compare before and after using partial correlation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ULTRA_HIGH_CORR = 0.998

print("Đang tính toán ma trận tương quan...")
corr_matrix = X_train.corr().abs()

ranker_temp = LGBMRegressor(n_estimators=100, verbose=-1, random_state=2601)
ranker_temp.fit(X_train, y_train)
importances = pd.Series(ranker_temp.feature_importances_, index=X_train.columns)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = []

for column in upper.columns:
    correlated_cols = upper.index[upper[column] > ULTRA_HIGH_CORR].tolist()
    
    if correlated_cols:
        for other_col in correlated_cols:
            if other_col in to_drop: continue
            
            imp_col = importances.get(column, 0)
            imp_other = importances.get(other_col, 0)
            
            if imp_col < imp_other:
                to_drop.append(column)
                break 
            else:
                to_drop.append(other_col)

to_drop = list(set(to_drop))
print(f"Đã tìm thấy {len(to_drop)} features trùng lặp (Corr > {ULTRA_HIGH_CORR})")

if len(to_drop) > 0:
    feats_filtered = [f for f in X_train.columns if f not in to_drop]
    
    print(f"Đang chạy đánh giá lại với {len(feats_filtered)} features...")
    res_filtered = get_metrics(f"Filtered (Corr > {ULTRA_HIGH_CORR})", feats_filtered)
    
    new_r2 = res_filtered['R2']
    
    old_r2 = df_res.loc[df_res['Method'] == 'Original', 'R2'].values[0]
    
    print(f"\n✅ Kết quả R2 Cũ (Original): {old_r2:.4f}")
    print(f"✅ Kết quả R2 Mới (Filtered): {new_r2:.4f}")
    
    methods = ['Original', 'Filtered']
    scores = [old_r2, new_r2]
    colors = ['gray', 'green' if new_r2 >= old_r2 else 'red']

    plt.figure(figsize=(8, 6))
    bars = plt.bar(methods, scores, color=colors, width=0.5)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.4f}',
                 ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.title(f'So sánh R2: Giữ nguyên vs Lọc Tương quan (> {ULTRA_HIGH_CORR})', fontsize=14)
    plt.ylabel('R2 Score (Real Scale)', fontsize=12)
    plt.ylim(0, max(scores) + 0.1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    new_row = pd.DataFrame([res_filtered])
    df_res = pd.concat([df_res, new_row], ignore_index=True)

else:
    print("✅ Không có features nào quá giống nhau để xóa.")

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')
df = df.dropna(subset=['Tm'])

df = df[(df['Tm'] > 0) & (df['Tm'] < 1000)].copy()

print(f"Data size after removing outliers (>1000K): {len(df)}")

y = df['Tm']
X = df.drop(columns=['Tm']).select_dtypes(include=[np.number])

y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

X.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='median')
X_final = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_log, test_size=0.2, random_state=2601)

corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.999)]

if to_drop:
    X_train = X_train.drop(columns=to_drop)
    X_test = X_test.drop(columns=to_drop)
    print(f"Dropped {len(to_drop)} high correlation features")

voting_model = LGBMRegressor(
    learning_rate=0.01,
    n_estimators=2000,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=-1,
    verbose=-1
)

voting_model.fit(X_train, y_train)

y_pred_log = voting_model.predict(X_test)
y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

new_r2 = r2_score(y_test_real, y_pred_real)
mae_real = mean_absolute_error(y_test_real, y_pred_real)
rmse_real = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("\n--- RESULT ---")
print(f"R2 Score : {new_r2:.4f}")
print(f"MAE      : {mae_real:.4f} K")
print(f"RMSE     : {rmse_real:.4f} K")

plt.figure(figsize=(10, 8))
plt.scatter(y_test_real, y_pred_real, alpha=0.4, color='blue', s=15, label='Data')

max_val = max(y_test_real.max(), y_pred_real.max())
min_val = min(y_test_real.min(), y_pred_real.min())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Fit')

plt.title(f'Evaluation (Tm < 1000K)\nR2 = {new_r2:.4f} | MAE = {mae_real:.2f} K', fontsize=14)
plt.xlabel('Actual Tm (K)', fontsize=12)
plt.ylabel('Predicted Tm (K)', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

---